#Setup

In [1]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.

In [2]:
import apache_beam as beam
import numpy as np
import time

#beam.Map

Gdy potrzebujesz wykonać prostą transformację na każdym elemencie (np. zmiana formatu, obliczenia, mapowanie).

In [9]:
dane_testowe = [
        {'id': i, 'wartosc': np.random.randint(1, 10), 'tekst': f"Hello world {i}"}
        for i in range(3)
    ]

dane_testowe

[{'id': 0, 'wartosc': 9, 'tekst': 'Hello world 0'},
 {'id': 1, 'wartosc': 8, 'tekst': 'Hello world 1'},
 {'id': 2, 'wartosc': 8, 'tekst': 'Hello world 2'}]

In [10]:
def beam_test():

    with beam.Pipeline() as pipeline:
        wynik_map = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'Map - Pomnożenie przez 2' >> beam.Map(lambda x: {'id': x['id'], 'wartosc': x['wartosc'] * 100, 'tekst': x['tekst']})
            | 'Wyświetlenie Map wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

{'id': 0, 'wartosc': 900, 'tekst': 'Hello world 0'}
{'id': 1, 'wartosc': 800, 'tekst': 'Hello world 1'}
{'id': 2, 'wartosc': 800, 'tekst': 'Hello world 2'}


#beam.FlatMap

Gdy chcesz rozdzielić dane (np. rozdzielenie linii tekstu na słowa).

In [17]:
dane_testowe = [
        {'id': i, 'wartosc': np.random.randint(1, 10), 'tekst': f"Hello world {i}"}
        for i in range(3)
    ]

dane_testowe

[{'id': 0, 'wartosc': 7, 'tekst': 'Hello world 0'},
 {'id': 1, 'wartosc': 8, 'tekst': 'Hello world 1'},
 {'id': 2, 'wartosc': 8, 'tekst': 'Hello world 2'}]

In [18]:
def beam_test():

    with beam.Pipeline() as pipeline:

        wynik_flatmap = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'FlatMap - Podział na słowa' >> beam.FlatMap(lambda x: x['tekst'].split())
            | 'Wyświetlenie FlatMap wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

Hello
world
0
Hello
world
1
Hello
world
2


#beam.Filter

Gdy potrzebujesz odfiltrować elementy na podstawie warunku.

In [25]:
dane_testowe = [
        {'id': i, 'wartosc': np.random.randint(1, 10), 'tekst': f"Hello world {i}"}
        for i in range(3)
    ]

dane_testowe

[{'id': 0, 'wartosc': 3, 'tekst': 'Hello world 0'},
 {'id': 1, 'wartosc': 7, 'tekst': 'Hello world 1'},
 {'id': 2, 'wartosc': 4, 'tekst': 'Hello world 2'}]

In [26]:
def beam_test():

    with beam.Pipeline() as pipeline:
        wynik_filter = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'Filter - Liczby parzyste' >> beam.Filter(lambda x: x['wartosc'] % 2 == 0)
            | 'Wyświetlenie Filter wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

{'id': 2, 'wartosc': 4, 'tekst': 'Hello world 2'}


#beam.CombinePerKey

Gdy dane są w postaci par (klucz, wartość) i chcesz połączyć wartości na podstawie kluczy (np. sumowanie, liczenie).

In [34]:
dane_testowe = [
    ('cat_1', np.random.randint(1, 1_000)),
    ('cat_2', np.random.randint(1, 1_000)),
    ('cat_1', np.random.randint(1, 100)),
    ('cat_2', np.random.randint(1, 100))
]

print(dane_testowe)
type(dane_testowe)

[('cat_1', 650), ('cat_2', 698), ('cat_1', 99), ('cat_2', 68)]


list

In [35]:
def beam_test():

    with beam.Pipeline() as pipeline:
        wynik_combine = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'CombinePerKey - Sumowanie wartości' >> beam.CombinePerKey(sum)
            | 'Wyświetlenie CombinePerKey wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

('cat_1', 749)
('cat_2', 766)


In [39]:
def beam_test():

    with beam.Pipeline() as pipeline:
        wynik_combine = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'CombinePerKey - Sumowanie wartości' >> beam.CombinePerKey(min)
            | 'Wyświetlenie CombinePerKey wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

('cat_1', 99)
('cat_2', 68)


#beam.GroupByKey

Gdy chcesz grupować elementy według kluczy, ale bez ich agregacji.

In [43]:
dane_testowe = [
    ('cat_1', np.random.randint(1, 1_000)),
    ('cat_2', np.random.randint(1, 1_000)),
    ('cat_1', np.random.randint(1, 100)),
    ('cat_2', np.random.randint(1, 100))
]

print(dane_testowe)

[('cat_1', 817), ('cat_2', 429), ('cat_1', 33), ('cat_2', 76)]


In [40]:
def beam_test():

    with beam.Pipeline() as pipeline:
        wynik_group_by_key = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'GroupByKey' >> beam.GroupByKey()
            | 'Wyświetlenie GroupByKey wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

('cat_1', [650, 99])
('cat_2', [698, 68])


#beam.GroupBy

Grupuje dane według klucza. Jest to przydatne w sytuacjach, w których chcesz połączyć elementy o tym samym kluczu, np. agregować dane.

In [47]:
dane_testowe = [
    ('cat_1', np.random.randint(1, 1_000)),
    ('cat_2', np.random.randint(1, 1_000)),
    ('cat_1', np.random.randint(1, 100)),
    ('cat_2', np.random.randint(1, 100))
]

print(dane_testowe)

[('cat_1', 752), ('cat_2', 915), ('cat_1', 32), ('cat_2', 12)]


In [46]:
def beam_test():

    with beam.Pipeline() as pipeline:
        wynik_group_by_key = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'GroupByKey' >> beam.GroupBy(lambda x: x[0])
            | 'Wyświetlenie GroupByKey wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

('cat_1', [('cat_1', 817), ('cat_1', 33)])
('cat_2', [('cat_2', 429), ('cat_2', 76)])


In [54]:
dane_testowe = [
    {'id': i, 'wartosc': np.random.randint(1, 100), 'kategoria': f'cat_{i % 2}'}
    for i in range(5)
]

dane_testowe

[{'id': 0, 'wartosc': 26, 'kategoria': 'cat_0'},
 {'id': 1, 'wartosc': 75, 'kategoria': 'cat_1'},
 {'id': 2, 'wartosc': 70, 'kategoria': 'cat_0'},
 {'id': 3, 'wartosc': 50, 'kategoria': 'cat_1'},
 {'id': 4, 'wartosc': 9, 'kategoria': 'cat_0'}]

In [55]:
def beam_test():

    with beam.Pipeline() as pipeline:
        wynik_group_by = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'GroupBy' >> beam.GroupBy(lambda x: x['kategoria'])
            | 'Wyświetlanie wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

('cat_0', [{'id': 0, 'wartosc': 26, 'kategoria': 'cat_0'}, {'id': 2, 'wartosc': 70, 'kategoria': 'cat_0'}, {'id': 4, 'wartosc': 9, 'kategoria': 'cat_0'}])
('cat_1', [{'id': 1, 'wartosc': 75, 'kategoria': 'cat_1'}, {'id': 3, 'wartosc': 50, 'kategoria': 'cat_1'}])


#beam.CombineGlobally

Gdy chcesz połączyć całą kolekcję w pojedynczą wartość (np. sumę, maksymalną wartość).

In [56]:
dane_testowe = [np.random.randint(1, 100) for _ in range(10)]
dane_testowe

[31, 47, 79, 70, 18, 63, 64, 40, 99, 69]

In [57]:
def beam_test():

    with beam.Pipeline() as pipeline:
        wynik_sumowanie = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'Sumowanie' >> beam.CombineGlobally(sum)
            | 'Wyświetlanie wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

580


# beam.Distinct()

Gdy chcesz zwrócić tylko unikalne elementy w kolekcji.

In [61]:
dane_testowe = [np.random.randint(1, 10) for _ in range(20)]
dane_testowe

[7, 6, 5, 9, 5, 8, 1, 3, 2, 8, 1, 3, 1, 7, 8, 1, 1, 5, 2, 4]

In [62]:
def beam_test():

    with beam.Pipeline() as pipeline:
        wynik_distinct = (
            pipeline
            | 'Tworzenie danych' >> beam.Create(dane_testowe)
            | 'Usuwanie duplikatów' >> beam.Distinct()
            | 'Wyświetlanie wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

7
6
5
9
8
1
3
2
4


#beam.Flatten()

Gdy masz wiele PCollection i chcesz je scalić.

In [65]:
dane_testowe1 = [1, 2, 3, 4, 5]
dane_testowe2 = [6, 7, 8, 9, 10]

print(dane_testowe1, dane_testowe2)

[1, 2, 3, 4, 5] [6, 7, 8, 9, 10]


In [68]:
def beam_test():

    with beam.Pipeline() as pipeline:
        kolekcja1 = pipeline | 'Tworzenie danych 1' >> beam.Create(dane_testowe1)
        kolekcja2 = pipeline | 'Tworzenie danych 2' >> beam.Create(dane_testowe2)

        wynik_flatten = (
            (kolekcja1, kolekcja2)  # Przekazujemy obie kolekcje w krotce
            | 'Połączenie kolekcji' >> beam.Flatten()
            | 'Wyświetlanie wyników' >> beam.Map(print)
        )

if __name__ == "__main__":
    beam_test()

1
2
3
4
5
6
7
8
9
10


#beam.Partition()

Pozwala na podział PCollection na dwie lub więcej części na podstawie funkcji warunkowej. Zastosujemy ją, aby rozdzielić dane na dwie grupy, np. liczby parzyste i nieparzyste.

In [70]:
def beam_test():

    dane_testowe = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

    def partition_function(element, num_partitions):
        return 0 if element % 2 == 0 else 1

    with beam.Pipeline() as pipeline:
        dane = pipeline | 'Tworzenie danych' >> beam.Create(dane_testowe)
        podzielone_dane = (
            dane
            | 'Partition danych' >> beam.Partition(partition_function, 2)
        )

        podzielone_dane[0] | 'Grupa parzyste' >> beam.Map(print)  # Grupa 0: Parzyste
        # podzielone_dane[1] | 'Grupa nieparzyste' >> beam.Map(print)  # Grupa 1: Nieparzyste

# Uruchomienie testu
if __name__ == "__main__":
    beam_test()

2
4
6
8
10


#beam.Reshuffle()

Gdy musisz rozproszyć dane, aby uniknąć przetwarzania ich w blokach.

In [71]:
dane_testowe = [
    ('k1', 1), ('k2', 2), ('k1', 3), ('k2', 4),
    ('k1', 5), ('k2', 6), ('k3', 7), ('k3', 8)
]

dane_testowe

[('k1', 1),
 ('k2', 2),
 ('k1', 3),
 ('k2', 4),
 ('k1', 5),
 ('k2', 6),
 ('k3', 7),
 ('k3', 8)]

In [77]:
import apache_beam as beam

def beam_test():

    with beam.Pipeline() as pipeline:
        dane = pipeline | 'Tworzenie danych' >> beam.Create(dane_testowe)
        pogrupowane_dane = (
            dane
            | 'GroupByKey' >> beam.GroupByKey()

        # ('k1', [1, 3, 5])
        # ('k2', [2, 4, 6])
        # ('k3', [7, 8])
        )

        reshuffled_dane = (
            pogrupowane_dane
            | 'Reshuffle' >> beam.Reshuffle()
        )

        reshuffled_dane | 'Wyświetlanie wyników' >> beam.Map(print)

if __name__ == "__main__":
    beam_test()

('k1', [1, 3, 5])
('k2', [2, 4, 6])
('k3', [7, 8])


In [82]:
def beam_test():
    with beam.Pipeline() as p:
        wynik = (p
                 | 'Tworzenie danych' >> beam.Create(dane_testowe)
                 | 'GroupByKey' >> beam.GroupByKey()
                 | 'Reshuffle' >> beam.Reshuffle()
                 | 'Wyświetlanie wyników' >> beam.Map(print)
                )

if __name__ == "__main__":
    beam_test()

('k1', [1, 3, 5])
('k2', [2, 4, 6])
('k3', [7, 8])


#beam.WindowInto()

####beam.WindowInto(beam.window.FixedWindows())

####beam.WindowInto(beam.window.SlidingWindows())

####beam.WindowInto(beam.window.SessionWindows())


1.	FixedWindows - Okna o stałej szerokości, np. 10 minut.
2.	SlidingWindows - Okna przesuwające się w czasie, np. okna co 5 minut, które zachodzą na siebie.
3.	SessionWindows - Okna o zmiennym czasie, tworzone w zależności od przerw w danych (np. jeśli dane nie napływają przez określony czas).

In [174]:
import random
import time

dane_testowe = [
    {
        "id": i + 1,
        "wartosc": random.random(),
        "kategoria": f"cat_{random.randint(0, 1)}",
        "timestamp": time.time() + (i * random.randint(5, 15)),
    }
    for i in range(200)
]

# for element in dane_testowe[:5]:
#     print(element)

In [177]:
from apache_beam import window

def beam_test():
    with beam.Pipeline() as pipeline:
        dane = pipeline | 'Tworzenie danych' >> beam.Create(dane_testowe)

        okienkowane_dane = (
            dane
            | 'Dodanie czasu' >> beam.Map(lambda x: beam.window.TimestampedValue((x['id'], x['wartosc'], x['kategoria']), x['timestamp']))

            # (1, 0.8208865302950905, 'cat_1')
            # (2, 0.4132579203461746, 'cat_0')
            # (3, 0.8967579354470947, 'cat_1')

            | 'Okna 100' >> beam.WindowInto(window.FixedWindows(3000))
            # | 'Okna przesuwne' >> beam.WindowInto(window.SlidingWindows(size=100, period=50))
            # | 'Okna 100' >> beam.WindowInto(window.Sessions(100))

            # (1, 0.8208865302950905, 'cat_1')
            # (2, 0.4132579203461746, 'cat_0')
            # (3, 0.8967579354470947, 'cat_1')
        )

        grupowanie_i_agregacja = (
            okienkowane_dane
            | 'Kluczowanie po kategorii' >> beam.Map(lambda x: (x[2], x[1]))

            # ('cat_1', 0.8208865302950905)
            # ('cat_0', 0.4132579203461746)
            # ('cat_1', 0.8967579354470947)


            | 'Grupowanie po kategorii' >> beam.GroupByKey()

            # ('cat_1', [0.8208865302950905, 0.8967579354470947, 0.40488050104897155, 0.25624277823085784, 0.9309313750956358, 0.13762827296879476, 0.8857082498141611, 0.4080801313041227, 0.2693505666917949, 0.8029403716771856, 0.16721599755841388, 0.8664963230135175, 0.006058765519106224, 0.6348193710941817, 0.7349182384862446, 0.5988226004736145, 0.5810971225830388, 0.4840414364917279, 0.3085641514763795, 0.04165458342389183, 0.11760090012631252, 0.1435366435017078, 0.9803522493151579, 0.937002784560225, 0.5203833943845655, 0.48977962225552063, 0.37718653771127464, 0.07529378705657797, 0.04188137092104727, 0.04287212077820035, 0.9504538627034936, 0.039079545154324435, 0.596265010035081, 0.48289492959341296, 0.1373577014104841, 0.5915893185521934, 0.3825988554161177, 0.9112290222085753, 0.3431882919579995, 0.5952882241839053, 0.24884246021443623, 0.05648683818762268, 0.6952983495813879, 0.5067949210695853, 0.8111031114966825, 0.3988546891442173, 0.9843854006278747, 0.398384458158731, 0.4112919434449749, 0.9467679807930265])
            # ('cat_1', [0.02382173607387006, 0.11679085670957512, 0.8490925431257342, 0.5184923089400104, 0.322527952656653, 0.1483100278318743, 0.29000123310302817, 0.6379760574992583, 0.09411857357372333, 0.47121887083006075, 0.9424315033655328, 0.6147381451183167, 0.34944599742226745, 0.3493348928033251, 0.6527947312582191, 0.5438206616743676, 0.054916949043656293, 0.34723663837519514, 0.18077068116232875, 0.852931395442046, 0.07565464667395572, 0.4081073354201147, 0.4544633869101937, 0.5950981245674081, 0.27444974495957464, 0.8704126615657269, 0.19643278991872748, 0.9099779926931029, 0.6913065838499685, 0.10137884907803718, 0.3395720897605645, 0.008770174857586244, 0.8581218058752492, 0.8484204750706928, 0.7309887133295168, 0.5237481580796451, 0.5817472408260147, 0.6524139869299961, 0.9614904913167214, 0.767304535443455, 0.8559049892149401, 0.8618831509541595, 0.9240782853711129, 0.7475434225433127, 0.2523251234112388, 0.5090851586716705, 0.48676778034839197, 0.5856978161900049, 0.7827682636141176, 0.6309127598139548])
            # ('cat_0', [0.4132579203461746, 0.4542661852741353, 0.10106765719908262, 0.5341264670927445, 0.3906231416084849, 0.1842329827690521, 0.617915256318081, 0.03336166691148579, 0.0379661291077511, 0.2764413673448034, 0.3908038849235922, 0.8582326116989138, 0.3243791020850988, 0.2664608740419603, 0.8508097977266957, 0.6320665971551271, 0.06979899200642836, 0.12645210868442514, 0.38372075009140416, 0.7207613141589994, 0.748768734132607, 0.8279338918944875, 0.3573649239680079, 0.6158273192586524, 0.3879516203709117, 0.2730709295295224, 0.38782596078220355, 0.886902364658491, 0.5278878574598045, 0.22077562632216197, 0.8592782942940868, 0.6969547118921814, 0.6504284957445998, 0.06848540034495021, 0.5907880135224097, 0.2364781034348752, 0.18853154857455423, 0.4002380704736901, 0.9111803420556258, 0.34868095843343294, 0.7623545529152449, 0.8593563159007251, 0.4409036772576135, 0.19306921443132508, 0.470033847785137, 0.09394741898323833, 0.49383052994884247, 0.5986271835707176, 0.18218564981557328])
            # ('cat_0', [0.5728238853525344, 0.4973198429764647, 0.2987110785132351, 0.4929952324678507, 0.45389069900050716, 0.9579469525535723, 0.8507834729936409, 0.5057478999701791, 0.9858698282838334, 0.5931988547450966, 0.5857324138107346, 0.3279193855841115, 0.3687512995069525, 0.21306331872895756, 0.6450349389058686, 0.8288620390225083, 0.06521984502714162, 0.7124561347511664, 0.5107416557510998, 0.39624916827895074, 0.5695944176724781, 0.9020552126930285, 0.3342397482744728, 0.7780120119249875, 0.08667280366160135, 0.7748296178950085, 0.3663690740428891, 0.3054798594221507, 0.30219698616791635, 0.7271084577787723, 0.7055166929682977, 0.8817827219138272, 0.9216045753447937, 0.7479017877522175, 0.6679819347577385, 0.29060929495478516, 0.7129941571497572, 0.5431081816905292, 0.6031136453261778, 0.4899924134301331, 0.9342024850330184, 0.003772182283846237, 0.705258235847913, 0.6325191640867852, 0.18772943561859945, 0.42139412293253486, 0.9473607335313563, 0.7373149864711038, 0.5799717544940668, 0.975554928371184])


            | 'Sumowanie w oknach' >> beam.Map(lambda x: (x[0], sum(x[1])))  # Sumowanie wartości w ramach grupy
        )

        grupowanie_i_agregacja | 'Wyświetlanie wyników' >> beam.Map(print)

if __name__ == "__main__":
    beam_test()

('cat_1', 13.189968853371731)
('cat_1', 32.40389653826267)
('cat_0', 22.62234987073723)
('cat_0', 33.31980960546236)


#beam.CoGroupByKey()

In [179]:
data_a = [
    ('key1', 'value_A1'),
    ('key2', 'value_A2'),
    ('key3', 'value_A3')
]

data_b = [
    ('key1', 'value_B1'),
    ('key2', 'value_B2'),
    ('key3', 'value_B3')
]

data_a[:2]

[('key1', 'value_A1'), ('key2', 'value_A2')]

In [180]:
def create_pcollections():
    with beam.Pipeline() as p:

        pc_a = (
            p
            | 'Create data A' >> beam.Create(data_a)
            | 'Map A' >> beam.Map(lambda x: (x[0], x[1]))
        )

        pc_b = (
            p
            | 'Create data B' >> beam.Create(data_b)
            | 'Map B' >> beam.Map(lambda x: (x[0], x[1]))
        )

        result = (
            {'A': pc_a, 'B': pc_b}
            | 'CoGroupByKey' >> beam.CoGroupByKey()
            | 'Format output' >> beam.Map(lambda x: f"Key: {x[0]}, A: {x[1]['A']}, B: {x[1]['B']}")
            | 'Print output' >> beam.Map(print)
        )


create_pcollections()

Key: key1, A: ['value_A1'], B: ['value_B1']
Key: key2, A: ['value_A2'], B: ['value_B2']
Key: key3, A: ['value_A3'], B: ['value_B3']


#beam.DoFn & beam.ParDo

- `beam.DoFn` pozwala na definiowanie niestandardowej logiki przetwarzania danych.
- `beam.ParDo` stosuje logikę zdefiniowaną w klasach DoFn do danych w potoku.

In [3]:
class FiltrujTransakcje(beam.DoFn):
    def __init__(self, minimalna_wartosc):
        self.minimalna_wartosc = minimalna_wartosc

    def process(self, element):
        if element['wartosc'] >= self.minimalna_wartosc:
            yield element

class PrzeksztalcTransakcje(beam.DoFn):
    def process(self, element):
        wynik = {
            'id': element['id'],
            'wartosc': round(element['wartosc'], 2)
        }
        yield wynik

dane_transakcyjne = [
    {"id": 1, "wartosc": 100.5, "kategoria": "A"},
    {"id": 2, "wartosc": 50.3, "kategoria": "B"},
    {"id": 3, "wartosc": 200.0, "kategoria": "A"},
    {"id": 4, "wartosc": 30.0, "kategoria": "C"},
    {"id": 5, "wartosc": 150.2, "kategoria": "B"},
]

minimalna_wartosc = 100.0

with beam.Pipeline() as pipeline:
    wyniki = (
        pipeline
        | "Tworzenie danych wejściowych" >> beam.Create(dane_transakcyjne)
        | "Filtruj transakcje" >> beam.ParDo(FiltrujTransakcje(minimalna_wartosc))
        | "Przekształć dane" >> beam.ParDo(PrzeksztalcTransakcje())
        | "Wypisz wyniki" >> beam.Map(print)
    )

{'id': 1, 'wartosc': 100.5}
{'id': 3, 'wartosc': 200.0}
{'id': 5, 'wartosc': 150.2}


#beam.CombineFn

- Klasa `CombineFn` do obliczania średniej

In [4]:
class ObliczSrednia(beam.CombineFn):
    def create_accumulator(self):
        # Inicjalizujemy akumulator w formie słownika: suma wartości i liczba elementów
        return {'suma': 0, 'liczba': 0}

    def add_input(self, accumulator, input):
        # Dodajemy wartość transakcji do akumulatora
        accumulator['suma'] += input['wartosc']
        accumulator['liczba'] += 1
        return accumulator

    def merge_accumulators(self, accumulators):
        # Łączymy wiele akumulatorów (w przypadku podziału na shardy)
        suma = sum(acc['suma'] for acc in accumulators)
        liczba = sum(acc['liczba'] for acc in accumulators)
        return {'suma': suma, 'liczba': liczba}

    def extract_output(self, accumulator):
        # Wyliczamy średnią z akumulatora (suma / liczba) i zwracamy wynik
        if accumulator['liczba'] == 0:
            return 0  # Obsługa przypadku braku danych
        return accumulator['suma'] / accumulator['liczba']

dane_transakcyjne = [
    {"id": 1, "wartosc": 100.5, "kategoria": "A"},
    {"id": 2, "wartosc": 50.3, "kategoria": "B"},
    {"id": 3, "wartosc": 200.0, "kategoria": "A"},
    {"id": 4, "wartosc": 30.0, "kategoria": "C"},
    {"id": 5, "wartosc": 150.2, "kategoria": "B"},
]

with beam.Pipeline() as pipeline:
    srednia_wartosc = (
        pipeline
        | "Tworzenie danych wejściowych" >> beam.Create(dane_transakcyjne)
        | "Oblicz średnią" >> beam.CombineGlobally(ObliczSrednia())  # Oblicz średnią dla całego zbioru
        | "Wypisz wynik" >> beam.Map(print)
    )

106.2


#beam.PTransform

In [ ]:
class FiltrujITransformuj(beam.PTransform):
    def __init__(self, prog):
        self.prog = prog

    def expand(self, pcoll):
        return (
            pcoll
            | "Filtruj transakcje" >> beam.Filter(lambda x: x["wartosc"] > self.prog)
            | "Przekształć na format docelowy" >> beam.Map(lambda x: {
                "id": x["id"],
                "wartosc": x["wartosc"]
            })
        )

dane_transakcyjne = [
    {"id": 1, "wartosc": 100.5, "kategoria": "A"},
    {"id": 2, "wartosc": 50.3, "kategoria": "B"},
    {"id": 3, "wartosc": 200.0, "kategoria": "A"},
    {"id": 4, "wartosc": 30.0, "kategoria": "C"},
    {"id": 5, "wartosc": 150.2, "kategoria": "B"},
]


prog_wartosci = 100.0


with beam.Pipeline() as pipeline:
    przetworzone_dane = (
        pipeline
        | "Tworzenie danych wejściowych" >> beam.Create(dane_transakcyjne)
        | "Filtruj i przekształć dane" >> FiltrujITransformuj(prog_wartosci)
        | "Wypisz wynik" >> beam.Map(print)
    )